# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 5시 (단, 발표자료는 11시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2020_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

# 데이터셋 다운로드
check = !if [ -d 'dataset/' ]; then echo "1" ; else echo "0"; fi
if (check[0] is '0' ):
  !mkdir dataset
  !wget 'https://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/matlab.zip'
  !unzip matlab.zip -d /content/dataset

# 데이터셋 로드
from scipy import io as spio
emnist = spio.loadmat("/content/dataset/matlab/emnist-balanced.mat")

x_train = emnist["dataset"][0][0][0][0][0][0]
y_train = emnist["dataset"][0][0][0][0][0][1]

x_test = emnist["dataset"][0][0][1][0][0][0]
y_test = emnist["dataset"][0][0][1][0][0][1]

# # 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 데이터 28x28 이미지화
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

# 총 클래스 개수
num_classes = y_test.shape[1]
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**



In [ ]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

# **3. 모델 생성**

In [ ]:
from keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

EPOCH = 150

# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# CNN (합성곱 신경망) 활용 레이어 제작
# glorot_uniform : vanshing gradient 문제를 해결하기 위한 가중치 초기화 
model.add(BatchNormalization())
model.add(Conv2D(100, (5,5), input_shape = (28,28,1), activation='relu', 
                 kernel_initializer='glorot_uniform'))

# Overfitting 방지를 위한 배치 정규화 및 드롭아웃 진행
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(80, (4,4), activation='relu', 
                 kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Conv2D(50, (3,3), activation='relu', 
                 kernel_initializer='glorot_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(50, (3,3), activation='relu', 
                 kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D())

# 1차원 텐서로 전환
model.add(Flatten())

model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform'))


# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 47, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))
# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = EPOCH, shuffle=True, callbacks=[cp_callback], validation_data=(x_test_after, y_test))

Epoch 1/150
879/882 [============================>.] - ETA: 0s - loss: 1.2179 - accuracy: 0.6430
Epoch 00001: val_accuracy improved from -inf to 0.81559, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 16s 19ms/step - loss: 1.2161 - accuracy: 0.6434 - val_loss: 0.5455 - val_accuracy: 0.8156
Epoch 2/150
880/882 [============================>.] - ETA: 0s - loss: 0.5361 - accuracy: 0.8181
Epoch 00002: val_accuracy improved from 0.81559 to 0.83287, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 16s 19ms/step - loss: 0.5361 - accuracy: 0.8181 - val_loss: 0.4738 - val_accuracy: 0.8329
Epoch 3/150
880/882 [============================>.] - ETA: 0s - loss: 0.4675 - accuracy: 0.8381
Epoch 00003: val_accuracy improved from 0.83287 to 0.85404, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 16s 19ms/step - loss: 0.4675 - accuracy: 0.8380 - val_loss: 0.4123 - v

# **4. 모델 저장**

In [ ]:
save_path = '/content/'
team_name = 'team11'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [ ]:
save_path = '/content/'
team_name = 'team11'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test_after, y_test)

588/588 [==============================] - 2s 3ms/step - loss: 0.3033 - accuracy: 0.8952


[0.3032628893852234, 0.8951596021652222]